In [1]:
import numpy as np
import pandas as pd
import nltk
import re

## 1. Load Data

In [2]:
df = pd.read_csv("corpus/tennis_articles.csv")

In [3]:
df.head()

,article_id,article_text,source
0,1,Maria Sharapova has basically no friends as te...,https://www.tennisworldusa.org/tennis/news/Mar...
1,2,"BASEL, Switzerland (AP), Roger Federer advance...",http://www.tennis.com/pro-game/2018/10/copil-s...
2,3,Roger Federer has revealed that organisers of ...,https://scroll.in/field/899938/tennis-roger-fe...
3,4,Kei Nishikori will try to end his long losing ...,http://www.tennis.com/pro-game/2018/10/nishiko...
4,5,"Federer, 37, first broke through on tour over ...",https://www.express.co.uk/sport/tennis/1036101...


In [9]:
df['article_text'][0]

"Maria Sharapova has basically no friends as tennis players on the WTA Tour. The Russian player has no problems in openly speaking about it and in a recent interview she said: 'I don't really hide any feelings too much. I think everyone knows this is my job here. When I'm on the courts or when I'm on the court playing, I'm a competitor and I want to beat every single person whether they're in the locker room or across the net.So I'm not the one to strike up a conversation about the weather and know that in the next few minutes I have to go and try to win a tennis match. I'm a pretty competitive girl. I say my hellos, but I'm not sending any players flowers as well. Uhm, I'm not really friendly or close to many players. I have not a lot of friends away from the courts.' When she said she is not really close to a lot of players, is that something strategic that she is doing? Is it different on the men's tour than the women's tour? 'No, not at all. I think just because you're in the same 

In [4]:
from nltk.tokenize import sent_tokenize
sentences = []
for text in df['article_text']:
    sentences += sent_tokenize(text)

In [6]:
# sentences

## Preprocess Data

In [7]:
from nltk.corpus import stopwords
from nltk import word_tokenize
stop_words = stopwords.words("english")

def is_ok(token):
    return re.search('^[a-z]+$', token) and token not in stop_words

def clean_sentence(sentence):
    return [word for word in word_tokenize(sentence.lower()) if is_ok(word)]

In [8]:
clean_sentence(sentences[0])

['maria',
 'sharapova',
 'basically',
 'friends',
 'tennis',
 'players',
 'wta',
 'tour']

In [9]:
clean_sentences = [clean_sentence(sentence) for sentence in sentences]

In [11]:
# clean_sentences

In [12]:
from gensim.models import KeyedVectors
filename = "GoogleNews-vectors-negative300.bin"
embedding = KeyedVectors.load_word2vec_format(filename, binary=True)

In [13]:
embedding['man']
# np.zeros((300,))

array([ 0.32617188,  0.13085938,  0.03466797, -0.08300781,  0.08984375,
       -0.04125977, -0.19824219,  0.00689697,  0.14355469,  0.0019455 ,
        0.02880859, -0.25      , -0.08398438, -0.15136719, -0.10205078,
        0.04077148, -0.09765625,  0.05932617,  0.02978516, -0.10058594,
       -0.13085938,  0.001297  ,  0.02612305, -0.27148438,  0.06396484,
       -0.19140625, -0.078125  ,  0.25976562,  0.375     , -0.04541016,
        0.16210938,  0.13671875, -0.06396484, -0.02062988, -0.09667969,
        0.25390625,  0.24804688, -0.12695312,  0.07177734,  0.3203125 ,
        0.03149414, -0.03857422,  0.21191406, -0.00811768,  0.22265625,
       -0.13476562, -0.07617188,  0.01049805, -0.05175781,  0.03808594,
       -0.13378906,  0.125     ,  0.0559082 , -0.18261719,  0.08154297,
       -0.08447266, -0.07763672, -0.04345703,  0.08105469, -0.01092529,
        0.17480469,  0.30664062, -0.04321289, -0.01416016,  0.09082031,
       -0.00927734, -0.03442383, -0.11523438,  0.12451172, -0.02

In [15]:
def get_word_vector(word):
    try:
        return embedding[word]
    except:
        return np.zeros((300,))

In [18]:
# get_word_vector('12345')

In [19]:
def get_sentence_vector(sentence):
    if len(sentence) != 0:
        word_vector_sum = sum([get_word_vector(word) for word in sentence])
        return word_vector_sum/len(sentence)
    return np.zeros((300,))

In [21]:
# get_sentence_vector(sentences[0])

In [22]:
sentence_vectors = [get_sentence_vector(sentence) for sentence in clean_sentences]

In [24]:
# sentence_vectors

## Similarity Matrix Preparation

In [25]:
from sklearn.metrics.pairwise import cosine_similarity

In [66]:
# sentence_vectors[0].reshape(1,-1)

In [26]:
cosine_similarity(sentence_vectors[0].reshape(1,-1),sentence_vectors[1].reshape(1,-1))

array([[0.47360238]])

In [27]:
def cal_cos_sim(sentence_vector1, sentence_vector2):
    return cosine_similarity(sentence_vector1.reshape(1,-1),sentence_vector2.reshape(1,-1))[0][0]

In [28]:
cal_cos_sim(sentence_vectors[0], sentence_vectors[1])

0.4736023752140091

In [30]:
sim_mat = np.zeros([len(sentences), len(sentences)])

In [31]:
for i in range(len(sentences)):
    for j in range(len(sentences)):
        if i != j:
            sim_mat[i][j] = cal_cos_sim(sentence_vectors[i], sentence_vectors[j])

In [32]:
sim_mat

array([[0.        , 0.47360238, 0.39921989, ..., 0.37955435, 0.37587219,
        0.33711867],
       [0.47360238, 0.        , 0.57306808, ..., 0.39835319, 0.42589647,
        0.45716587],
       [0.39921989, 0.57306808, 0.        , ..., 0.40601331, 0.3769902 ,
        0.48700741],
       ...,
       [0.37955435, 0.39835319, 0.40601331, ..., 0.        , 0.35059899,
        0.43938726],
       [0.37587219, 0.42589647, 0.3769902 , ..., 0.35059899, 0.        ,
        0.37479851],
       [0.33711867, 0.45716587, 0.48700741, ..., 0.43938726, 0.37479851,
        0.        ]])

## Applying PageRank Algorithm

In [33]:
import networkx as nx

nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)

In [34]:
# scores

In [35]:
sentence_scores = [(scores[i],s) for i,s in enumerate(sentences)]
sentence_scores

[(0.008430560782176615,
  'Maria Sharapova has basically no friends as tennis players on the WTA Tour.'),
 (0.008277735614673834,
  "The Russian player has no problems in openly speaking about it and in a recent interview she said: 'I don't really hide any feelings too much."),
 (0.008091799925889454, 'I think everyone knows this is my job here.'),
 (0.010516167119469877,
  "When I'm on the courts or when I'm on the court playing, I'm a competitor and I want to beat every single person whether they're in the locker room or across the net.So I'm not the one to strike up a conversation about the weather and know that in the next few minutes I have to go and try to win a tennis match."),
 (0.007364453945133579, "I'm a pretty competitive girl."),
 (0.007225629774569331,
  "I say my hellos, but I'm not sending any players flowers as well."),
 (0.009313029709688175,
  "Uhm, I'm not really friendly or close to many players."),
 (0.00750861092265802, "I have not a lot of friends away from the 

In [36]:
ranked_sentences = sorted(sentence_scores, reverse=True)
ranked_sentences

[(0.010516167119469877,
  "When I'm on the courts or when I'm on the court playing, I'm a competitor and I want to beat every single person whether they're in the locker room or across the net.So I'm not the one to strike up a conversation about the weather and know that in the next few minutes I have to go and try to win a tennis match."),
 (0.010262652786438639,
  'Speaking at the Swiss Indoors tournament where he will play in Sundays final against Romanian qualifier Marius Copil, the world number three said that given the impossibly short time frame to make a decision, he opted out of any commitment.'),
 (0.010141756916913726,
  'Major players feel that a big event in late November combined with one in January before the Australian Open will mean too much tennis and too little rest.'),
 (0.010077819273090738,
  '"I felt like the best weeks that I had to get to know players when I was playing were the Fed Cup weeks or the Olympic weeks, not necessarily during the tournaments.'),
 (0.

In [37]:
for i in range(10):
    print(ranked_sentences[i][1])

When I'm on the courts or when I'm on the court playing, I'm a competitor and I want to beat every single person whether they're in the locker room or across the net.So I'm not the one to strike up a conversation about the weather and know that in the next few minutes I have to go and try to win a tennis match.
Speaking at the Swiss Indoors tournament where he will play in Sundays final against Romanian qualifier Marius Copil, the world number three said that given the impossibly short time frame to make a decision, he opted out of any commitment.
Major players feel that a big event in late November combined with one in January before the Australian Open will mean too much tennis and too little rest.
"I felt like the best weeks that I had to get to know players when I was playing were the Fed Cup weeks or the Olympic weeks, not necessarily during the tournaments.
Kei Nishikori will try to end his long losing streak in ATP finals and Kevin Anderson will go for his second title of the ye